<a href="https://colab.research.google.com/github/goelnikhils-lgtm/languagemodels/blob/main/GeneratingStruturedOutput.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install instructor pydantic google-generativeai jsonref

In [ ]:
import instructor
import google.generativeai as genai
from pydantic import BaseModel, Field
from enum import Enum
from typing import Literal

In [ ]:
GOOGLE_API_KEY = ""
genai.configure(api_key="GOOGLE_API_KEY")

In [ ]:
#pyndatic model def
class Person(BaseModel):
  name:str = Field(max_length= 50)
  age:int | str
  role:str = Field(description ="Role of the person in company , ex:developer")
  company:Literal["google","microsoft","meta"]

class PeopleExtraction(BaseModel):
  reasoning:str = Field(description ="Based on the provided information , write 'how' will you approach extracting information from the text.")
  people_list: list[Person] = Field(min_length= 3)

In [ ]:
#initialize gemini client
client = instructor.from_gemini(
    client = genai.GenerativeModel(model_name ="models/gemini-2.0-flash"),
    model = instructor.Mode.GEMINI_JSON,
)

In [ ]:
#input text
text =""" Thomas , 34 years old is an aritifical intelligence engineer at google.
he spends his day fine tuning models in collaboration with Nikhil , 52 years old , data sceientist at meta,
a company that is doing cutting edge AI research.
Nikhil ensures he inteacts intutitively with models fine tuned with companies like Thomas
"""


In [ ]:
#prepare the message to extract people
message = [
    {
        "role":"user",
        "content":f"Extract all people mentioned in this text:{text}"
    }

]

In [ ]:
#Extracting people as a generator
people_extraction = client.chat.completions.create(
    messages = message,
    response_model = PeopleExtraction,
    max_tries = 2
)

In [ ]:
#display the reasoning and the names and roles of the extracted people
print("Model Reasoning:\n ")
print(people_extraction.reasoning)
print("\nList of extracted people:\n")
for i, person in enumerate(people_extraction.people_list, start =1):
  print(f" Name :{person.name} , Age:{person.age} years old , Role:{person.role} at {person.company}")
